In [5]:
# 05_human_in_loop.py
import os
import json
import re
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool

load_dotenv()

# ============================================================
# CONFIGURATION
# ============================================================

QUALITY_THRESHOLD = 4.0
JUDGE_CONFIDENCE_THRESHOLD = 0.8
MAX_ITERATIONS = 3

search_tool = SerperDevTool()

# ============================================================
# AGENTS
# ============================================================

researcher = Agent(
    role="Senior Research Analyst",
    goal="Find the most relevant academic papers on a topic",
    backstory="""You are a meticulous researcher who finds high-quality 
    academic sources from top venues like arXiv, NeurIPS, ICML, and ACL.
    You prioritize recent, well-cited papers.""",
    tools=[search_tool],
    verbose=True
)

summarizer = Agent(
    role="Technical Writer",
    goal="Create clear, accurate summaries of research papers",
    backstory="""You are a skilled technical writer who distills complex 
    research into accessible insights. When given feedback, you incorporate 
    it thoughtfully to improve your work.""",
    verbose=True
)

critic = Agent(
    role="Quality Reviewer",
    goal="Evaluate summaries for accuracy, clarity, and completeness",
    backstory="""You are a demanding but fair editor with high standards.
    You provide numerical scores AND specific actionable feedback.""",
    verbose=True
)

judge = Agent(
    role="Senior Editor",
    goal="Make final publication decisions based on overall quality",
    backstory="""You are the final gatekeeper before publication. You assess 
    whether content meets publication standards holistically—considering 
    accuracy, clarity, completeness, and whether it serves the reader's needs. 
    You assign a confidence score reflecting your certainty that this content 
    is ready for publication. You are conservative: when in doubt, flag for 
    human review.""",
    verbose=True
)

# ============================================================
# HELPER FUNCTIONS
# ============================================================

def extract_scores(critique_output):
    """Parse the critic's JSON output to extract average score."""
    try:
        json_match = re.search(r'\[.*\]', critique_output, re.DOTALL)
        if json_match:
            evaluations = json.loads(json_match.group())
            scores = []
            for eval_item in evaluations:
                avg = (eval_item.get('accuracy', 0) + 
                       eval_item.get('clarity', 0) + 
                       eval_item.get('completeness', 0)) / 3
                scores.append(avg)
            return sum(scores) / len(scores) if scores else 0
    except (json.JSONDecodeError, KeyError, TypeError) as e:
        print(f"⚠️  Could not parse critique scores: {e}")
    return 0

def extract_feedback(critique_output):
    """Extract written feedback from critique for next iteration."""
    try:
        json_match = re.search(r'\[.*\]', critique_output, re.DOTALL)
        if json_match:
            evaluations = json.loads(json_match.group())
            feedback_items = [eval_item.get('feedback', '') for eval_item in evaluations]
            if any(feedback_items):
                return "Previous feedback to address:\n- " + "\n- ".join(f for f in feedback_items if f)
    except (json.JSONDecodeError, KeyError, TypeError):
        pass
    return ""

# ============================================================
# PHASE 1: RESEARCH
# ============================================================

def run_research_phase(topic):
    """Phase 1: Research papers on the given topic."""
    
    research_task = Task(
        description=f"""Search for the top 3 most relevant academic papers on: {topic}
        
        For each paper, provide:
        - Title (exact)
        - Authors (first author et al. for papers with many authors)
        - Year and venue
        - Key findings (verbatim abstract or close paraphrase)
        
        Focus on influential, well-cited papers from reputable venues.
        """,
        expected_output="A structured list of 3 papers with full metadata and findings.",
        agent=researcher
    )
    
    research_crew = Crew(
        agents=[researcher],
        tasks=[research_task],
        process=Process.sequential,
        verbose=True
    )
    
    result = research_crew.kickoff()
    return result.raw

# ============================================================
# PHASE 2: SUMMARIZE + CRITIQUE LOOP
# ============================================================

def run_summarize_critique_loop(research_results):
    """Phase 2: Iteratively summarize and critique until quality threshold is met."""
    
    previous_feedback = ""
    final_summaries = ""
    avg_score = 0
    
    for iteration in range(1, MAX_ITERATIONS + 1):
        print(f"\n📝 Iteration {iteration}/{MAX_ITERATIONS}: Summarizing and critiquing...")
        
        # Build the summarize task with any previous feedback
        feedback_section = f"\n\nIMPORTANT - Address this feedback from the previous iteration:\n{previous_feedback}" if previous_feedback else ""
        
        summarize_task = Task(
            description=f"""Based on the research below, create a summary for each paper.
            
            For each paper, write:
            - A 2-3 sentence summary of the main contribution
            - The key methodology or approach used
            - The primary results or findings
            - One sentence on broader implications
            
            Write for a technical audience. Be precise but accessible.
            {feedback_section}
            
            RESEARCH TO SUMMARIZE:
            {research_results}
            """,
            expected_output="Three paper summaries with contribution, methodology, results, and implications.",
            agent=summarizer
        )
        
        critique_task = Task(
            description="""Review the summaries and evaluate each one.
            
            For each summary, rate on a 1-5 scale:
            - **accuracy**: Does it correctly represent the paper's claims?
            - **clarity**: Is it understandable to the target audience?
            - **completeness**: Are the key points covered?
            
            Return your evaluation as a JSON array:
```json
            [
                {"paper": "Paper Title", "accuracy": X, "clarity": X, "completeness": X, 
                 "feedback": "Specific suggestions for improvement"}
            ]
```
            
            Be precise in your scores. 5 means excellent, 3 means acceptable, 1 means poor.
            Always provide specific, actionable feedback.
            """,
            expected_output="JSON array with scores and feedback for each summary.",
            agent=critic
        )
        
        # Run summarizer + critic
        refine_crew = Crew(
            agents=[summarizer, critic],
            tasks=[summarize_task, critique_task],
            process=Process.sequential,
            verbose=True
        )
        
        result = refine_crew.kickoff()
        final_summaries = result.raw
        
        # Extract scores from the critique portion
        avg_score = extract_scores(final_summaries)
        print(f"\n📊 Iteration {iteration} average score: {avg_score:.2f}")
        
        if avg_score >= QUALITY_THRESHOLD:
            print(f"✅ Quality threshold ({QUALITY_THRESHOLD}) met!")
            break
        
        if iteration < MAX_ITERATIONS:
            previous_feedback = extract_feedback(final_summaries)
            print(f"⚠️  Score {avg_score:.2f} below threshold ({QUALITY_THRESHOLD}). Refining...")
        else:
            print(f"🚨 Max iterations reached. Proceeding with current output.")
    
    return final_summaries, avg_score

# ============================================================
# PHASE 3: JUDGE EVALUATION
# ============================================================

def run_judge_evaluation(summaries):
    """Phase 3: Run the judge to evaluate final summaries."""
    
    judge_task = Task(
        description=f"""Review the final summaries and provide a publication decision.
        
        SUMMARIES TO REVIEW:
        {summaries}
        
        For each summary, evaluate:
        1. Is the information accurate and well-supported?
        2. Is the writing clear and professional?
        3. Does it serve the reader's needs?
        
        Return your judgment as JSON:
```json
        {{
            "overall_confidence": 0.XX,
            "recommendation": "publish" or "revise" or "human_review",
            "paper_evaluations": [
                {{"paper": "Title", "confidence": 0.XX, "issues": "Any concerns or empty string"}}
            ],
            "general_comments": "Overall assessment"
        }}
```
        
        Confidence scale:
        - 0.9-1.0: Excellent, publish immediately
        - 0.8-0.9: Good, acceptable for publication
        - 0.7-0.8: Adequate, consider minor revisions
        - Below 0.7: Needs work, recommend human review
        """,
        expected_output="JSON with confidence scores and publication recommendation.",
        agent=judge
    )
    
    judge_crew = Crew(
        agents=[judge],
        tasks=[judge_task],
        process=Process.sequential,
        verbose=True
    )
    
    print("\n⚖️  Running Judge evaluation...")
    result = judge_crew.kickoff()
    
    # Parse judge output
    try:
        json_match = re.search(r'\{.*\}', result.raw, re.DOTALL)
        if json_match:
            judgment = json.loads(json_match.group())
            confidence = judgment.get('overall_confidence', 0)
            recommendation = judgment.get('recommendation', 'unknown')
            
            print(f"\n📊 Judge confidence: {confidence:.2%}")
            print(f"📋 Recommendation: {recommendation}")
            
            return confidence, recommendation, judgment
    except (json.JSONDecodeError, KeyError) as e:
        print(f"⚠️  Could not parse judge output: {e}")
    
    return 0, "error", {}

# ============================================================
# PHASE 4: HUMAN-IN-THE-LOOP
# ============================================================

def get_human_approval(content, context=""):
    """Request human approval for content before proceeding."""
    
    print("\n" + "="*60)
    print("🔔 HUMAN REVIEW REQUESTED")
    print("="*60)
    
    if context:
        print(f"\n📋 Context: {context}")
    
    print("\n" + "-"*40)
    print("CONTENT FOR REVIEW:")
    print("-"*40)
    print(content)
    print("-"*40 + "\n")
    
    while True:
        response = input("Approve this content? [y]es / [n]o / [e]dit with feedback: ").lower().strip()
        
        if response in ['y', 'yes']:
            print("✅ Approved by human reviewer.")
            return True, None
        elif response in ['n', 'no']:
            feedback = input("Please provide feedback for revision: ").strip()
            print("❌ Rejected by human reviewer.")
            return False, feedback
        elif response in ['e', 'edit']:
            feedback = input("Enter your feedback: ").strip()
            print(f"📝 Feedback recorded: {feedback}")
            return False, feedback
        else:
            print("⚠️  Please enter 'y' (yes), 'n' (no), or 'e' (edit with feedback)")

# ============================================================
# COMPLETE PIPELINE WITH HUMAN-IN-THE-LOOP
# ============================================================

def run_pipeline_with_hitl(topic):
    """Full pipeline with human-in-the-loop approval."""
    
    print(f"\n{'='*60}")
    print(f"🔬 RESEARCH ASSISTANT PIPELINE (with Human-in-the-Loop)")
    print(f"📚 Topic: {topic}")
    print(f"{'='*60}")
    
    # Phase 1: Research
    print("\n" + "="*40)
    print("📚 PHASE 1: Research")
    print("="*40)
    research_results = run_research_phase(topic)
    print("\n✅ Research phase complete.")
    
    # Phase 2: Summarize + Critique Loop
    print("\n" + "="*40)
    print("📝 PHASE 2: Summarization & Critique")
    print("="*40)
    summaries, critique_score = run_summarize_critique_loop(research_results)
    print(f"\n✅ Summarization phase complete. Final score: {critique_score:.2f}")
    
    # Phase 3: Judge Evaluation
    print("\n" + "="*40)
    print("⚖️  PHASE 3: Judge Evaluation")
    print("="*40)
    confidence, recommendation, judgment = run_judge_evaluation(summaries)
    
    # Phase 4: Determine if human review is needed
    print("\n" + "="*40)
    print("👤 PHASE 4: Human Review Decision")
    print("="*40)
    
    needs_human_review = (
        confidence < JUDGE_CONFIDENCE_THRESHOLD or
        recommendation == "human_review"
    )
    
    if needs_human_review:
        print(f"\n⚠️  Human review required:")
        print(f"   - Judge confidence: {confidence:.2%} (threshold: {JUDGE_CONFIDENCE_THRESHOLD:.2%})")
        print(f"   - Recommendation: {recommendation}")
        
        context = f"Judge confidence: {confidence:.2%} | Recommendation: {recommendation}"
        approved, feedback = get_human_approval(summaries, context)
        
        if approved:
            return summaries, "approved_by_human"
        else:
            print(f"\n📝 Human feedback for revision: {feedback}")
            return summaries, "rejected_by_human"
    else:
        # High confidence - offer optional human review
        print(f"\n✅ Judge confidence ({confidence:.2%}) meets threshold.")
        print("   Content is ready for publication.")
        
        skip_review = input("\nSkip human review and auto-publish? [y/n]: ").lower().strip()
        
        if skip_review in ['y', 'yes']:
            print("✅ Auto-approved based on high judge confidence.")
            return summaries, "auto_approved"
        else:
            approved, feedback = get_human_approval(summaries, f"Judge confidence: {confidence:.2%}")
            if approved:
                return summaries, "approved_by_human"
            else:
                print(f"\n📝 Human feedback: {feedback}")
                return summaries, "rejected_by_human"

# ============================================================
# MAIN
# ============================================================

if __name__ == "__main__":
    topic = "large language model alignment"
    
    final_content, status = run_pipeline_with_hitl(topic)
    
    print("\n" + "="*60)
    print(f"🏁 FINAL STATUS: {status.upper()}")
    print("="*60)
    
    if "approved" in status:
        print("\n📰 Publishing article...")
        print("-"*40)
        print(final_content)
        print("-"*40)
        print("\n✅ Article published!")
    else:
        print("\n📋 Article saved as draft for revision.")
        print("-"*40)
        print(final_content)
        print("-"*40)

NameError: name 'run_research_and_summarize' is not defined